<a href="https://colab.research.google.com/github/DongWooLeee/Tobigs19/blob/main/week3_DT_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DT Assignment

# Data Loading

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기


- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [ ]:
def get_gini(df, label):
    
    gini=1
    p_js = df[label].value_counts() / len(df)
    for p_j in p_js:
      gini-=p_j**2
        
    return gini

In [ ]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [ ]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    result=[]
    #0개,1개, 2개,3개...len(uniques)
    for i in range(len(uniques) - 1): # -1해줘야 마지막에 (1,len(uniques)-1) 개까지 생성됨)
      for j in list(combinations(uniques, i+1)):
          result.append(list(j))
    return result

In [ ]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [ ]:
pd_data.columns

Index(['age', 'income', 'student', 'credit_rating', 'class_buys_computer'], dtype='object')

In [ ]:
pd_data.class_buys_computer.unique()

array(['no', 'yes'], dtype=object)

In [ ]:
pd_data.income.unique()

array(['high', 'medium', 'low'], dtype=object)

In [ ]:
pd_data['class_buys_computer']

0      no
1      no
2     yes
3     yes
4     yes
5      no
6     yes
7      no
8     yes
9     yes
10    yes
11    yes
12    yes
13     no
Name: class_buys_computer, dtype: object

In [ ]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    results = get_binary_split(df, attribute)#해당 attribute 상에서 취할수 있는 모든 이진분류 경우의수
    n = len(df)
    for res in results: 
        idx = '_'.join(res)#출력값처럼해주기 위함
        df_0 = df[df[attribute].isin(res)] 
        df_1 = df[~df[attribute].isin(res)] # 이진분류에 따른 데이터프레임 두개로 분리
        gini_index = len(df_0) / n * get_gini(df_0, label) + len(df_1) / n * get_gini(df_1, label)
        result[idx] = gini_index
    
    return result

In [ ]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [ ]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

In [ ]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [ ]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    min_gini = min(get_attribute_gini_index(pd_data,feature,'class_buys_computer').items())
    print(f'Minimum Gini index of {feature} : {min_gini}')

Minimum Gini index of age : ('middle_aged', 0.35714285714285715)
Minimum Gini index of income : ('high', 0.4428571428571429)
Minimum Gini index of student : ('no', 0.3673469387755103)
Minimum Gini index of credit_rating : ('excellent', 0.42857142857142855)


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [ ]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [ ]:
from math import log2

def getEntropy(df, feature) :
  entropy = 0
  probs = df[feature].value_counts()/len(df)
  
  for prob in probs:
    entropy+=(-prob*log2(prob))
    ### entropy summation
  return entropy


In [ ]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [ ]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}
#가장 먼저 label에 대한 entropy get
    ent = getEntropy(df, feature)
    cols = df.columns
    cols=cols.drop(feature)
#각 column의 고유값을 살펴보고 각 class 집합에 대한 entropy 구하기
    for col in cols:
      col_ent = 0
      for val in df[col].unique():#각 칼럼의 고윳값에 대하여 class 집합에 대한 entropy 산출
        S_val = sum(df[col]==val)
        S = len(df)
        col_ent += (S_val/S)*getEntropy(df[df[col]==val],feature)
      result[col] = ent-col_ent #한 칼럼의 entropy 값 구하기
    
    return result
  


        

In [ ]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}